## Decision Tree and Random Forest
Decision trees are a type of regression technique that aims to discern some set of discrete features from a data set.  Decision trees function are built from a subset of branches (specific features) and nodes (where the branches split).

Random Forest ...

To do membrane detection in Ilastik, I followed the carving technique.  Carving refers to separating individual objects by their boundaries (which is unlike what classification does -- classification looks at the individual brightness values to try to determine differences in the internal structure, while carving functions primarily by examining borders).

I used their suggested mouse retina EM data courtesy Winfried Dink et al.

